<a href="https://colab.research.google.com/github/Jagdish9903/Intel-Project-TeleICU/blob/main/Patient_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#download human-action-classification-video-dataset
!kaggle datasets download -d sharjeelmazhar/human-activity-recognition-video-dataset -p /content/drive/MyDrive/intel_proj/

Dataset URL: https://www.kaggle.com/datasets/sharjeelmazhar/human-activity-recognition-video-dataset
License(s): CC-BY-NC-SA-4.0
100% 14.8G/14.8G [13:27<00:00, 21.0MB/s]
100% 14.8G/14.8G [13:27<00:00, 19.7MB/s]


In [ ]:
#install mediapipe package for keypoints detection
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
#install yt-dlp to download youtube videos for testing
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
#unzip the dataset
import zipfile
import os

dataset_path = '/content/drive/MyDrive/intel_proj/human-activity-recognition-video-dataset.zip'
unzip_path = '/content/drive/MyDrive/intel_proj'

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

In [ ]:
#import all dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dropout, Dense
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model for key-points detection
mp_drawing = mp.solutions.drawing_utils # Drawing utilities to draw skeletons

In [ ]:
#takes mediapipe model and video frame and outputs results and frame as it is
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
#better visualizations of keypoints imposed on frames
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:

#create single vector for each frame detected keypoints and take zero if not detected
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
# Directory where your dataset is stored
DATASET_PATH = os.path.join('drive','MyDrive','intel_proj','Human Activity Recognition - Video Dataset')

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('drive','MyDrive','intel_proj','MP_Data')

# Actions that we try to detect
actions = np.array(['Clapping', 'Sitting', 'Standing Still','Walking'])

In [ ]:
#save extracted keypoints in .npy file to be used as dataset for classification model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in actions:
        action_path = os.path.join(DATASET_PATH, action)
        videos = os.listdir(action_path)[:70]  # List the first 70 video files in the action directory
        # Loop through each video file
        for sequence, video_file in enumerate(videos):
            video_path = os.path.join(action_path, video_file)
            cap = cv2.VideoCapture(video_path)
            frame_num = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                os.makedirs(os.path.dirname(npy_path), exist_ok=True)  # Create directory if it doesn't exist
                np.save(npy_path, keypoints)

                frame_num += 1

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            cap.release()
cv2.destroyAllWindows()

In [ ]:
results #visualize mediapipe output

mediapipe.python.solution_base.SolutionOutputs

In [ ]:
results.pose_landmarks

landmark {
  x: 0.63159281
  y: 0.180676371
  z: -0.532001793
  visibility: 0.999732673
}
landmark {
  x: 0.65620482
  y: 0.167847484
  z: -0.509443343
  visibility: 0.999354422
}
landmark {
  x: 0.669236779
  y: 0.168853045
  z: -0.509680033
  visibility: 0.999407053
}
landmark {
  x: 0.678176701
  y: 0.169411361
  z: -0.509903193
  visibility: 0.999448836
}
landmark {
  x: 0.620464087
  y: 0.16438815
  z: -0.478037238
  visibility: 0.99902451
}
landmark {
  x: 0.609624743
  y: 0.163290769
  z: -0.477934211
  visibility: 0.998809576
}
landmark {
  x: 0.598394156
  y: 0.162302405
  z: -0.47779581
  visibility: 0.998620629
}
landmark {
  x: 0.700261354
  y: 0.177263081
  z: -0.303221792
  visibility: 0.999623299
}
landmark {
  x: 0.591125131
  y: 0.168195903
  z: -0.157532528
  visibility: 0.996293724
}
landmark {
  x: 0.651857376
  y: 0.203220874
  z: -0.447876185
  visibility: 0.999778807
}
landmark {
  x: 0.608891428
  y: 0.200544417
  z: -0.406840533
  visibility: 0.999315
}
landmar

In [ ]:
label_map = {label:num for num, label in enumerate(actions)} #create label maping for training

In [ ]:
label_map

{'Clapping': 0, 'Sitting': 1, 'Standing Still': 2, 'Walking': 3}

In [ ]:
#load saved dataset of .npy files
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(30):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            if os.path.isfile(file_path):
                res = np.load(file_path)
                window.append(res)
        if len(window) == 30:  # Ensure only complete windows are added
            sequences.append(window)
            labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

(280, 30, 1662)

In [ ]:
np.array(labels).shape

(280,)

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

(280, 30, 1662)

In [ ]:
#one-hot encode labels
y = tf.keras.utils.to_categorical(labels).astype(int)

In [ ]:
X_val.shape

(55, 200, 1662)

In [ ]:
y_val.shape

(55, 4)

In [ ]:
#tensorboard configuration to look over training process
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
!tensorboard --logdir=Logs

2024-07-07 15:58:25.004003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 15:58:25.004115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 15:58:25.157970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 15:58:28.152768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to 

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
#create model architecture
model = Sequential()
model.add(LSTM(100, input_shape=(30,1662)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
#compile mode with necessary parameters
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#start training
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
9/9 [==============================] - 4s 182ms/step - loss: 1.4346 - accuracy: 0.2782 - val_loss: 1.2665 - val_accuracy: 0.4286
Epoch 2/50
9/9 [==============================] - 1s 97ms/step - loss: 1.3041 - accuracy: 0.3571 - val_loss: 1.1406 - val_accuracy: 0.5000
Epoch 3/50
9/9 [==============================] - 1s 119ms/step - loss: 1.2677 - accuracy: 0.3684 - val_loss: 0.9964 - val_accuracy: 0.5714
Epoch 4/50
9/9 [==============================] - 2s 203ms/step - loss: 1.2218 - accuracy: 0.4173 - val_loss: 1.0110 - val_accuracy: 0.5000
Epoch 5/50
9/9 [==============================] - 3s 299ms/step - loss: 1.2137 - accuracy: 0.4398 - val_loss: 1.0234 - val_accuracy: 0.7143
Epoch 6/50
9/9 [==============================] - 2s 250ms/step - loss: 1.2446 - accuracy: 0.3910 - val_loss: 1.0133 - val_accuracy: 0.7143
Epoch 7/50
9/9 [==============================] - 2s 184ms/step - loss: 1.2001 - accuracy: 0.4135 - val_loss: 0.9819 - val_accuracy: 0.7143
Epoch 8/50
9/9 [=====

In [ ]:
#save model
model.save('action_30.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#predict on test data
yhat = model.predict(X_test)

1/1 [==============================] - 0s 468ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
#validate with confusion matrix
multilabel_confusion_matrix(ytrue, yhat)

array([[[10,  0],
        [ 0,  4]],

       [[10,  1],
        [ 0,  3]],

       [[10,  0],
        [ 1,  3]],

       [[11,  0],
        [ 0,  3]]])

In [ ]:
#test accuracy
accuracy_score(ytrue, yhat)

0.9285714285714286

In [ ]:
#download test video from youtube
!yt-dlp -f mp4 -o "download_walk.mp4" "https://youtube.com/shorts/LnYTAGmgphs?si=qXNH00sq4PkH3ncN"


[youtube] Extracting URL: https://youtube.com/shorts/LnYTAGmgphs?si=qXNH00sq4PkH3ncN
[youtube] LnYTAGmgphs: Downloading webpage
[youtube] LnYTAGmgphs: Downloading ios player API JSON
[youtube] LnYTAGmgphs: Downloading m3u8 information
[info] LnYTAGmgphs: Downloading 1 format(s): 18
[download] download_walk.mp4 has already been downloaded
[download] 100% of    1.15MiB


In [ ]:
#visualize the predicted class and its probability on output video
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

In [ ]:
#load video
model = tf.keras.models.load_model('/content/action_30.h5')

In [ ]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245),(255,0,0)]

res = None
image = None
results = None
action = None

# Function to save video
def save_video(input_path, output_path, model, actions, threshold=0.8):
    # Initialize MediaPipe holistic model
    mp_holistic = mp.solutions.holistic

    # Capture video from the provided file path
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Initialize sequence and sentence variables
    sequence = []
    sentence = []

    with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
        frame_idx=0
        media_idx = 0
        while cap.isOpened():
            global results
            global image
            ret, frame = cap.read()
            if not ret:
                break

            # Make detections
            if media_idx % 4 == 0:   #skip frames and transfer output of previous detection on undetected frames
              image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:   #take 30 frames window for prediction
                global res
                global action
                if frame_idx % 4 == 0:
                  res = model.predict(np.expand_dims(sequence, axis=0))[0]
                  action = actions[np.argmax(res)]

                # Viz logic
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if action != sentence[-1]:
                            sentence.append(action)
                    else:
                        sentence.append(action)

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)

                frame_idx += 1

            # Add the sentence to the frame
            cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Write the frame to the output video
            out.write(image)

            media_idx += 1

        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Path to input and output videos
input_video_path = '/content/download_walk.mp4'
output_video_path = 'vid_walk_gpu8.mp4'

# Call the function to process the video
save_video(input_video_path, output_video_path, model, actions)

1/1 [==============================] - 0s 18ms/step
